## **Software :** Threading

#### _Les exécutions parallèles en Python_

🟠 `on work`

---

1. **Principe**
    * Programmation linéaire
    * Programmation parallèle
2. **Les Threads**
    * Concept
    * Synchronisation
3. **Asynchronisation**
    * .

`---`

* [Real Python :: Getting Started](https://realpython.com/python-async-features/)
* [Real Python :: Async IO](https://realpython.com/async-io-python/)


**Built-in**

In [41]:
import time
from threading import Thread, RLock

---
### **1.** Principe

##### **1.1** - Programmation linéaire

In [21]:
# Un script bloqué pendant (au moins) 2 secondes
print("Éxécuté avant '.sleep'")
time.sleep(2)
print("Éxécuté après '.sleep' : ± 2 secondes d'attente")

Éxécuté avant '.sleep'
Éxécuté après '.sleep' : ± 2 secondes d'attente


Code bloquant

In [20]:
# 5 itérations éxécutées en linéaire : ± 10 secondes nécessaires
def thread_function(num) :
    print(f"Thread {num} : Start")
    time.sleep(2)
    print(f"Thread {num} : End")

for i in range(5) :
    thread_function(i)
    print("\t=> Fin de la boucle")

Thread 0 : Start
Thread 0 : End
	=> Fin de la boucle
Thread 1 : Start
Thread 1 : End
	=> Fin de la boucle
Thread 2 : Start
Thread 2 : End
	=> Fin de la boucle
Thread 3 : Start
Thread 3 : End
	=> Fin de la boucle
Thread 4 : Start
Thread 4 : End
	=> Fin de la boucle


##### **1.2** - Programmation parallèle

La classe `Thread`

In [16]:
# La classe Thread
# help(Thread)
print(Thread.__doc__)

A class that represents a thread of control.

    This class can be safely subclassed in a limited fashion. There are two ways
    to specify the activity: by passing a callable object to the constructor, or
    by overriding the run() method in a subclass.

    


In [17]:
# Méthode de lancement d'un Thread
print(Thread.start.__doc__)

Start the thread's activity.

        It must be called at most once per thread object. It arranges for the
        object's run() method to be invoked in a separate thread of control.

        This method will raise a RuntimeError if called more than once on the
        same thread object.

        


In [18]:
# Méthode d'exécution liée à un Thread
print(Thread.run.__doc__)

Method representing the thread's activity.

        You may override this method in a subclass. The standard run() method
        invokes the callable object passed to the object's constructor as the
        target argument, if any, with sequential and keyword arguments taken
        from the args and kwargs arguments, respectively.

        


Code non-bloquant

In [11]:
# 5 itérations exécutées en parallèle : ± 2 secondes seulement
class ThreadFunction(Thread) :

    def __init__(self, name) :
        Thread.__init__(self)
        self.name = name
    
    # (!) - Surcharge de la fonction run() nécessaire
    def run(self) :
        print(f"Thread {self.name} : Start")
        time.sleep(2)
        print(f"Thread {self.name} : End")

for i in range(5) :
    thread = ThreadFunction(i)
    thread.start()
    print("\t=> Fin de la boucle\n")

Thread 0 : Start
	=> Fin de la boucle

Thread 1 : Start
	=> Fin de la boucle

Thread 2 : Start
	=> Fin de la boucle

Thread 3 : Start
	=> Fin de la boucle

Thread 4 : Start
	=> Fin de la boucle



Thread 1 : End
Thread 2 : End
Thread 4 : End
Thread 0 : End
Thread 3 : End


---
### **2.** Les Threads

##### **2.1** - Concept

In [37]:
# Construction d'un Thread d'écriture dans un fichier
class ClassicThreading(Thread) :

    def __init__(self, text:str) -> None :
        Thread.__init__(self)
        self.__text = text

    def run(self) -> None :
        print(f"Texte : {self.__text}")
        with open('./_classic_threads.txt', 'a') as file :
            file.write(f"{self.__text}\n")

In [38]:
# Mise en place de plusieurs Theads pour écrire dans un fichier
thread_0 = ClassicThreading('Thread #0 :: Text content... ')
thread_1 = ClassicThreading('Thread #1 :: Another text content... ')
thread_2 = ClassicThreading('Thread #2 :: Text content, again... ')
thread_3 = ClassicThreading('Thread #3 :: Text... ')
thread_4 = ClassicThreading('Thread #4 :: And text... ')

thread_0.start()
thread_1.start()
thread_2.start()
thread_3.start()
thread_4.start()

Texte : Thread #0 :: Text content... 
Texte : Thread #1 :: Another text content... 
Texte : Thread #2 :: Text content, again... 
Texte : Thread #3 :: Text... 
Texte : Thread #4 :: And text... 


In [39]:
# Observation du résultat...
classic_threads = open('./_classic_threads.txt')
print(classic_threads.read())

Thread #0 :: Text content... 
Thread #2 :: Text content, again... 
Thread #1 :: Another text content... 
Thread #3 :: Text... 
Thread #4 :: And text... 



Résultat après une 3<sup>e</sup> exécution des Threads :
```
Thread #0 :: Text content... 
Thread #2 :: Text content, again... 
Thread #1 :: Another text content... 
Thread #3 :: Text... 
Thread #4 :: And text... 
```
Le **Thread #2** a été exécuté **avant le Thread #1** (ce n'est pas toujours le cas). Aussi, lorsque plusieurs _Threads_ accèdent à une même ressource, par exemple une base de données, l'ordre d'exécution ne peut pas être garantit. Les _Threads_ s'exécutent en parallèle, mais ne se terminent pas toujours dans l'ordre de lancement initial.

##### **2.2** - Synchronisation

Objet `RLock`

In [43]:
# Bloquer l'exécution d'un autre Thread à une même ressource
print(RLock.__doc__)

Factory function that returns a new reentrant lock.

    A reentrant lock must be released by the thread that acquired it. Once a
    thread has acquired a reentrant lock, the same thread may acquire it again
    without blocking; the thread must release it once for each time it has
    acquired it.

    


In [46]:
# Thread contrôlé
class SyncThreading(Thread) :

    def __init__(self, text:str, lock:RLock) -> None :
        Thread.__init__(self)
        self.__text = text
        self.__lock = lock

    def run(self) -> None :
        print(f"Texte : {self.__text}")
        with self.__lock :
            with open('./_sync_threads.txt', 'a') as file :
                file.write(f"{self.__text}\n")

In [47]:
# Lancement
lock = RLock()
sync_thread_0 = SyncThreading('SyncThread #0 :: Text content... ', lock)
sync_thread_1 = SyncThreading('SyncThread #1 :: Another text content... ', lock)
sync_thread_2 = SyncThreading('SyncThread #2 :: Text content, again... ', lock)
sync_thread_3 = SyncThreading('SyncThread #3 :: Text... ', lock)
sync_thread_4 = SyncThreading('SyncThread #4 :: And text... ', lock)

sync_thread_0.start()
sync_thread_1.start()
sync_thread_2.start()
sync_thread_3.start()
sync_thread_4.start()

Texte : SyncThread #0 :: Text content... 
Texte : SyncThread #1 :: Another text content... 
Texte : SyncThread #2 :: Text content, again... 
Texte : SyncThread #3 :: Text... 
Texte : SyncThread #4 :: And text... 


Méthode `.join()`

In [44]:
# Attendre la fin d'un Thread en cours d'exécution
print(Thread.join.__doc__)

Wait until the thread terminates.

        This blocks the calling thread until the thread whose join() method is
        called terminates -- either normally or through an unhandled exception
        or until the optional timeout occurs.

        When the timeout argument is present and not None, it should be a
        floating point number specifying a timeout for the operation in seconds
        (or fractions thereof). As join() always returns None, you must call
        is_alive() after join() to decide whether a timeout happened -- if the
        thread is still alive, the join() call timed out.

        When the timeout argument is not present or None, the operation will
        block until the thread terminates.

        A thread can be join()ed many times.

        join() raises a RuntimeError if an attempt is made to join the current
        thread as that would cause a deadlock. It is also an error to join() a
        thread before it has been started and attempts to do so ra

---
### **3.** Asynchronisation